In [40]:
import soundfile as sf
import torchaudio
from pesq import pesq
import random
import os
import numpy as np
import pickle
from collections import Counter
from pprint import pprint

### noise type statistics

In [2]:
def get_noise_stat(noise_type_list):
    noise_type = np.array(list(Counter(noise_type_list).keys()))
    count =  np.array(list(Counter(noise_type_list).values()))
    type_count = np.rec.fromarrays((noise_type, count), names=('noise_type', 'count'))
    pprint(type_count[np.argsort(-1*count)].tolist())

In [3]:
root = '/workspace/data/train'
train_noise_type_list = []

for flac_name in os.listdir(root):
    if flac_name.endswith('.flac'):
        if flac_name.split('_')[0] == 'mixed':
            train_noise_type_list.append(flac_name.split('_')[-1].split('.')[0])
print('train_noise_type_stat:')
get_noise_stat(train_noise_type_list)

train_noise_type_stat:
[('music', 4462),
 ('shot', 2832),
 ('conditioner', 2830),
 ('playing', 2819),
 ('horn', 2817),
 ('idling', 2816),
 ('drilling', 2802),
 ('jackhammer', 1677),
 ('siren', 1674),
 ('cleaner', 1672),
 ('fan', 1672),
 ('rainy', 1671),
 ('market', 1669),
 ('blower', 1667),
 ('train', 1665),
 ('truck', 1664),
 ('traffic', 1656),
 ('bark', 1651),
 ('grinding', 1640),
 ('silence', 472)]


In [53]:
root = '/workspace/data/test'
train_noise_type_list = []

for flac_name in os.listdir(root):
    if flac_name.endswith('.flac'):
        if flac_name.split('_')[0] == 'mixed':
            train_noise_type_list.append(flac_name.split('_')[-1].split('.')[0])
print('test_noise_type_stat:')
get_noise_stat(train_noise_type_list)

test_noise_type_stat:
[('music', 90),
 ('train', 63),
 ('rainy', 61),
 ('market', 61),
 ('grinding', 60),
 ('truck', 59),
 ('cleaner', 59),
 ('siren', 59),
 ('jackhammer', 58),
 ('blower', 56),
 ('traffic', 56),
 ('bark', 56),
 ('fan', 56),
 ('playing', 35),
 ('conditioner', 33),
 ('idling', 33),
 ('drilling', 33),
 ('horn', 33),
 ('shot', 32),
 ('silence', 7)]


### Get path for different noise type 

In [45]:
noise_type = np.array(list(Counter(train_noise_type_list).keys()))
init_list = [[] for _ in noise_type]
noise_type_dict = dict(zip(noise_type, init_list))

root = '/workspace/data/train'

for flac_name in os.listdir(root):
    if flac_name.endswith('.flac'):
        if flac_name.split('_')[0] == 'mixed':
            noise_type_dict[flac_name.split('_')[-1].split('.')[0]].append(flac_name)

with open('/workspace/SE_2022/train_all_noise__by_type.pkl', 'wb') as f:
    pickle.dump(noise_type_dict, f)

train_path_dict = {}
val_path_dict = {}

for key in list(noise_type_dict.keys()):
    random.seed(0)
    path_array = np.array(noise_type_dict[key])
    num_of_sample = path_array.shape[0]
    val_index = random.sample(range(num_of_sample), int(num_of_sample*0.1))
    val_path_dict[key] = path_array[val_index]
    train_path_dict[key] = np.delete(path_array, val_index)

with open('/workspace/SE_2022/train_noise_by_type.pkl', 'wb') as f:
    pickle.dump(train_path_dict, f)

with open('/workspace/SE_2022/val_noise_by_type.pkl', 'wb') as f:
    pickle.dump(val_path_dict, f)

### Get the map of noise file to clean file mapping for train data

In [14]:
root = '/workspace/data/train'
train_map = {}

for flac_name in os.listdir(root):
    if flac_name.endswith('.flac'):
        if flac_name.split('_')[0] == 'mixed':
            number = flac_name.split('_')[1]
            clean_file_name = 'vocal_'+ number + '.flac'
            noise_file_path = os.path.join(root, flac_name)
            clean_file_path = os.path.join(root, clean_file_name)
            train_map[noise_file_path] = clean_file_path

In [16]:
len(train_map)

41828

In [19]:
with open('/workspace/SE_2022/train_map.pkl', 'wb') as f:
    pickle.dump(train_map, f)

In [2]:
with open('/workspace/SE_2022/train_map.pkl', 'rb') as f:
    train_map = pickle.load(f)

In [3]:
train_map

{'/workspace/data/train/mixed_01001_cleaner.flac': '/workspace/data/train/vocal_01001.flac',
 '/workspace/data/train/mixed_01002_siren.flac': '/workspace/data/train/vocal_01002.flac',
 '/workspace/data/train/mixed_01003_fan.flac': '/workspace/data/train/vocal_01003.flac',
 '/workspace/data/train/mixed_01004_traffic.flac': '/workspace/data/train/vocal_01004.flac',
 '/workspace/data/train/mixed_01005_music.flac': '/workspace/data/train/vocal_01005.flac',
 '/workspace/data/train/mixed_01006_dog_bark.flac': '/workspace/data/train/vocal_01006.flac',
 '/workspace/data/train/mixed_01007_rainy.flac': '/workspace/data/train/vocal_01007.flac',
 '/workspace/data/train/mixed_01008_train.flac': '/workspace/data/train/vocal_01008.flac',
 '/workspace/data/train/mixed_01009_siren.flac': '/workspace/data/train/vocal_01009.flac',
 '/workspace/data/train/mixed_01010_jackhammer.flac': '/workspace/data/train/vocal_01010.flac',
 '/workspace/data/train/mixed_01011_car_horn.flac': '/workspace/data/train/vocal